In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import pygame
import pandas as pd
import sys
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class ArrayStack:
    '''
    LIFO栈
    用于定义和改变试管中水排序
    操作复杂度均为O（1）
    '''
    def __init__(self,items:"list")->list:
        # 创建一个空栈
        self._data = items
    
    def __len__(self):
        # 返回栈中的元素数量
        return len(self._data)

    def isEmpty(self):
        # 如果栈为空则返回“True”
        return len(self._data) == 0
    
    def push(self,a):
        # 向栈顶添加元素a
        self._data.append(a)

    def pop(self):
        # 移除栈顶的元素并返回，如果栈为空，则返回一个错误
        if self.isEmpty():
            raise Exception('Stack is empty')
        return self._data.pop()

    def top(self):
        # 返回栈顶的元素但不移除，如果栈为空，则返回一个错误
        if self.isEmpty():
            raise Exception('Stack is empty')
        return self._data[-1]
    
    def sum(self):
        #计算全栈元素和
        return sum(self._data)
        
    def regulate(self,regulation):
        #调整栈顶的元素
        self._data[-1]=self._data[-1]-regulation
    def remove_last_record(self):
        #移除栈顶的元素（回溯栈时使用）
        if self.isEmpty():
            raise Exception('Stack is empty')
        self._data.pop()
    def record_new_branch(self,a):
        #将这次分支记录在已有的该层历史决策中
        self._data[-1].append(a)


class tube:
    '''
    用两个stack分别记录水的序列和对应体积
    tube:
        state check > isEmpty ; isFull ; issorted ; 
        pour_into
        
    '''
    def __init__(self,order:"ArrayStack",Vols:"ArrayStack",Max_Vol):
        self.order=order
        self.Vols=Vols
        self.Max_Vol=Max_Vol
        
    def isEmpty(self):
        #返回试管是否为空
        return self.order.isEmpty()
    
    def isFull(self):
        #返回试管是否已满
        return self.Vols.sum()==self.Max_Vol
    
    def issorted(self):
        #返回该试管是否完成排序，即是否仅有一种颜色并且装满
        if len(self.order)==1 and self.isFull():
            return True
        else:
            return False
        
    def pour_into(self,Tube:"tube")->"end event":
        '''
        判断能否倒水:空试管或未装满且颜色相同
        完成倒水动作
        '''
        if not ( Tube.isFull() or self.isEmpty() ) :
            WaterType=self.order.top()
            vol=self.Vols.top()
            remain_space=self.Max_Vol-Tube.Vols.sum()
            if Tube.isEmpty():
                self.order.pop()
                self.Vols.pop()
                Tube.order.push(WaterType)
                Tube.Vols.push(vol)
            else:
                if Tube.order.top()==WaterType:
                    if vol>remain_space:       
                        self.Vols.regulate(remain_space)
                        Tube.Vols.regulate(-remain_space)
                    else:
                        self.order.pop()
                        self.Vols.pop()
                        Tube.Vols.regulate(-vol) 
                else:
                    pass

class environment:
    def __init__(self,nonempty_num,empty_num,Max_Vol,difficulty,colornum):
        self.nonempty_num=nonempty_num #n
        self.empty_num=empty_num #k
        self.Max_Vol=Max_Vol
        self.colornum=colornum #m种颜色
        self.difficulty=difficulty
        
    def initialize(self):
        '''
        随机生成最初状态：
        共有n个非空试管，从排序结果开始，
        首先切片，混合，再打乱，再切片
        '''
        Mixorder=[]
        MixVols=[]
        
        #切片混合
        for i in range(self.colornum):
            pieces_num=random.randint(1,self.difficulty)
            colorcode=i
            cut_points=[random.random()*self.Max_Vol for i in range(pieces_num-1)]
            cut_points.sort()
            pieces=[]
            i_1=0
            for i in cut_points:
                pieces.append(i-i_1)
                i_1=i
            pieces.append(self.Max_Vol-i_1)
            Mixorder+=[colorcode]*len(pieces)
            MixVols+=pieces
            print('pieces',pieces,colorcode)

        if self.nonempty_num>self.colornum:
            
            for i in range(self.nonempty_num-self.colornum):
                pieces_num=random.randint(1,self.difficulty)
                colorcode=random.randint(0,self.colornum-1)
                cut_points=[random.random()*self.Max_Vol for i in range(pieces_num-1)]
                cut_points.sort()
                pieces=[]
                i_1=0
                for i in cut_points:
                    pieces.append(i-i_1)
                    i_1=i
                pieces.append(self.Max_Vol-i_1)
                Mixorder+=[colorcode]*len(pieces)
                MixVols+=pieces
                print('pieces',pieces,colorcode)
            
        #打乱
        cc=list(zip(Mixorder,MixVols))
        random.shuffle(cc)
        Mixorder[:], MixVols[:] = zip(*cc)
        
        #合并相邻相同的液体
        ad_order=ArrayStack([])
        ad_Vols=ArrayStack([])
        for i,j in zip(Mixorder,MixVols):
            
            if ad_order.isEmpty():
                ad_order.push(i)
                ad_Vols.push(j)
            
            else:
                if i==ad_order.top():
                    ad_Vols.regulate(-j)
                else:
                    ad_order.push(i)
                    ad_Vols.push(j)
        
        print('adjoint:',ad_order._data,ad_Vols._data)
        #切片   
        summ=0
        single_order=[]
        single_vol=[]
        orders=[]
        vols=[]
        for i,j in zip(ad_order._data,ad_Vols._data):
            if summ+j>self.Max_Vol:
                
                single_order.append(i)
                single_vol.append(self.Max_Vol-summ)
                orders.append(single_order)
                vols.append(single_vol)
                summ=j-(self.Max_Vol-summ)
                single_order=[i]
                single_vol=[summ]
                
            elif summ+j<self.Max_Vol:
                summ=summ+j  
                single_order.append(i)
                single_vol.append(j)
            else:
                single_order.append(i)
                single_vol.append(j)
                orders.append(single_order)
                vols.append(single_vol)
                summ=0
                single_order=[]
                single_vol=[]
        
        tubes=[]
        for i,j in zip(orders,vols):
            tubes.append(tube(ArrayStack(i),ArrayStack(j),self.Max_Vol))
        for i in range(self.empty_num):
            tubes.append(tube(ArrayStack([]),ArrayStack([]),self.Max_Vol))
        self.tubes=tubes
        return tubes
    
    def game_cheat(self):
        self.status=0
        self.history=ArrayStack([])
        self.solutions=ArrayStack([])
        self.rec=ArrayStack([])
        self.added_empty_tubes=0
        self.recursive_time=0
        # copy_env=self
        self.calculation=[]
        self.anwser=self.search_for_answer()

    def search_for_answer(self):
        if self.status==0 or self.status==2:  ##第一次遇见这种环境
            state_useful,empty_tube,state_single = self.state_greedy1()
            current_solutions = self.strategy_greedy1(state_useful,empty_tube,state_single)  ## 求解目前所有可行解
            calculate_non_empty=0
            calculate_empty=0
            for tube in self.tubes:
                if tube.issorted():
                    continue
                else:
                    if tube.isEmpty():
                        calculate_empty+=1
                    else:
                        calculate_non_empty+=1
            self.calculation.append((calculate_empty,calculate_non_empty))
            if current_solutions==[]:   ##找不到可行解
                sortednum=0
                for tube in self.tubes:
                    if tube.issorted():
                        sortednum+=1
                if sortednum==self.nonempty_num:   ##判断胜利与否
                    return(self.rec)    ##输出last step集合，即完全解法
                else:
                    self.status=1  ##转换为状态1
                    self.go_back() ##回到上一试管状态
                    return self.search_for_answer() ##继续下一步
            else:                 ##目前有解
                self.status=0   ##转换为状态0
                self.history.push(self.tubes) ### 记录此刻的状态,以供回溯
                self.solutions.push(current_solutions) #####记录该层全部可行解
                current_operation=current_solutions[0] #####执行最贪心的策略
                self.rec.push(current_operation)  ###记录决策
                for i in current_operation:
                    self.pour(i)       ####执行最优解
                return self.search_for_answer() ##继续下一步
        else:          ##通过回溯来到这个状态
            if self.rec.top()==self.solutions.top()[-1]: ######过去已经走过了该层所有决策
                self.status=3  ##转换为状态3
                self.history.remove_last_record() ###此状态以后不会再回来了
                self.solutions.remove_last_record()  ###此层可行解已用尽
                self.rec.remove_last_record()     ###此条路径已经完全否定
                self.go_back() ##回到上一试管状态
                return self.search_for_answer() ##继续下一步
            else:   ####这层的可行解还没用完
                self.status=2 ##转换为状态2
                current_operation=self.solutions.top()[self.solutions.top().index(self.rec.top())+1]  ####索引移到次一位贪心，准备执行
                self.rec.push(current_operation) ###记录决策
                for i in current_operation:
                    self.pour(i)       ####执行最优解
                return self.search_for_answer() ##继续下一步
    def go_back(self):
        if self.history.isEmpty():
            print('empty tube number:',self.empty_num,'is not enough')
            self.empty_num+=1
            self.added_empty_tubes+=1
            self.status=0
            self.search_for_answer() 
        self.tubes=self.history.top() ####  试管重置到上一状态
    def pour(self,from_to):
        self.tubes[from_to[0]].pour_into(self.tubes[from_to[1]])
    def state_greedy1(self):
        unempty_state = [[i,self.tubes[i].order._data[-1],self.tubes[i].Vols.top(),1-sum(self.tubes[i].Vols._data)]  for i in range(len(self.tubes)) if self.tubes[i].isEmpty() == False]
        unempty_state = pd.DataFrame(unempty_state)
        unempty_state.columns = ['tube','top_color','top_vol','top_emp_vol']
        unempty_state = unempty_state[unempty_state['top_vol']!=1]#去除满的试管
        empty_tube = [i for i in range(len(self.tubes)) if self.tubes[i].isEmpty() == True]
        colors = unempty_state.groupby('top_color')['tube'].describe()['count']
        color_chosen = colors[colors.values>1].index
        color_single = colors[colors.values==1].index
        state_useful = unempty_state.loc[unempty_state['top_color'].isin(color_chosen)]
        state_single = unempty_state.loc[unempty_state['top_color'].isin(color_single)]
        return state_useful,empty_tube,state_single

    def strategy_greedy1(self,state_useful,empty_tube,state_single):
        strategies = []
        Rep_dict = {x:state_useful[state_useful['top_color'].isin([x])] for x in state_useful['top_color'].unique()}#判断并分开颜色
        for color in Rep_dict:
            df = Rep_dict[color]
            df.sort_values(by="top_emp_vol", inplace=True, ascending=False)#按空余体积降序排序
            df_sort_em = df.copy()
            df.sort_values(by="top_vol", inplace=True, ascending=True)#按最上层体积升序排序
            df_sort_top = df.copy()
            #若该瓶仅有一种颜色，则优先，由于对于此种情况，倒不完是有意义的
            pure = []
            pure_stra = []
            if len(df[df['top_vol']+df['top_emp_vol']==1]["tube"])>0:
                for i in df[df['top_vol']+df['top_emp_vol']==1]["tube"]:
                    pure.append(i)
                    df_s = df_sort_top[df_sort_top['tube']!=i]
                    begain,end = 0,1
                    ind = df_s.index
                    while end<len(ind):#由于此处一种颜色至少有两个，故除去一个的话不会少于一个，不会出现end>len(ind)
                        begain = end-1 #由于不同的是否被倒完对于环境意义不同，故需-1
                        aval_vol = df_sort_top[df_sort_top['tube']==i]['top_emp_vol'][df_sort_top.index[0]]
                        while df_s['top_vol'][ind[begain:end]].sum()<=aval_vol:
                            end+=1
                        pure_stra.append([(tp,i) for tp in df_s['tube'][ind[begain:end]]]) 
            #对于瓶中不仅有一种颜色的情况，相互倾倒，若倒不完则没有意义，按最小液体体积倒入最大空余体积来贪心排序
            strategies.extend(pure_stra)
            com_stra = []
            df_sor_em = df_sort_em[~df_sort_em['tube'].isin(pure)]
            ind_em,ind_to = df_sor_em.index,df_sort_top.index
            if df_sor_em.loc[ind_em[0]]['top_emp_vol']>=df_sort_top.loc[ind_to[0]]['top_vol']:#避免无用尝试
                for ip in ind_em:#index与tube号一致
                    daf_sort = df_sort_top[df_sort_top['tube']!=ip]
                    ind_t = daf_sort.index
                    vol_em = df_sor_em.loc[ip]['top_emp_vol']
                    en = 0
                    while vol_em>=daf_sort.loc[ind_t[en]]['top_vol'] and en<len(ind_t):#由于已排序好，不满足的之后不用尝试
                        be = en
                        while df_s['top_vol'][ind[be:en]].sum()<=vol_em:
                            en+=1
                        com_stra.append([(tp,ip) for tp in ind_t[be:en-1]]) 
            strategies.extend(com_stra)
        strategies=sorted(strategies,key=len)
        strategies.reverse()
        if len(empty_tube)>0:
            stra_tog,stra_single = [],[]
            #多个倒空试管
            Rep_num = {x:len(Rep_dict[x]) for x in Rep_dict}#计数
            Rep_num = sorted( Rep_num.items(),key=lambda x:x[1],reverse=True)#排序
            for j in Rep_num:
                #可全部倒进去
                if Rep_dict[j[0]]['top_vol'].sum()<1:
                    stra_tog.append([(tu,empty_tube[0]) for tu in Rep_dict[j[0]]['tube']])
                #只能倒一部分进去，按贪心每次组合尽可能倒多的
                else:
                    begain,end = 0,1
                    ind = Rep_dict[j[0]].index
                    while end<len(ind):
                        begain = end-1 #由于不同的是否被倒完对于环境意义不同，故需-1
                        while Rep_dict[j[0]]['top_vol'][ind[begain:end]].sum()<=1:
                            end+=1
                        stra_tog.append([(tu,empty_tube[0]) for tu in Rep_dict[j[0]]['tube'][ind[begain:end]]])
            #单独倒单个进入空试管
            state_single.sort_values(by="top_vol", inplace=True, ascending=False)#排序
            stra_single = [[(i,empty_tube[0])] for i in state_single['tube']]
            strategies.extend(stra_tog)
            strategies.extend(stra_single)
        return strategies

import pygame

display_width = 1200
display_height = 600

WHITE = (255, 255, 255)
RED = (255, 0, 0)
bg_location = 'pic1.jpg'

pygame.init()

class Button(object):
    def __init__(self, text, color, x=None, y=None, **kwargs):
        self.surface = font.render(text, True, color)

        self.WIDTH = self.surface.get_width()
        self.HEIGHT = self.surface.get_height()

        if 'centered_x' in kwargs and kwargs['centered_x']:
            self.x = display_width // 2 - self.WIDTH // 2
        else:
            self.x = x

        if 'centered_y' in kwargs and kwargs['cenntered_y']:
            self.y = display_height // 2 - self.HEIGHT // 2
        else:
            self.y = y

    def display(self):
        screen.blit(self.surface, (self.x, self.y))

    def check_click(self, position):
        x_match = position[0] > self.x and position[0] < self.x + self.WIDTH
        y_match = position[1] > self.y and position[1] < self.y + self.HEIGHT

        if x_match and y_match:
            return True
        else:
            return False
        
class InputBox:
    def __init__(self, rect: pygame.Rect = pygame.Rect(100, 100, 140, 32)) -> None:
        """
        rect，传入矩形实体，传达输入框的位置和大小
        """
        self.boxBody: pygame.Rect = rect
        self.color_inactive = pygame.Color('lightskyblue3')  # 未被选中的颜色
        self.color_active = pygame.Color('dodgerblue2')  # 被选中的颜色
        self.color = self.color_inactive  # 当前颜色，初始为未激活颜色
        self.active = False
        self.text = ''
        self.done = False
        self.font = pygame.font.Font(None, 32)

    def dealEvent(self, event: pygame.event.Event):
        if(event.type == pygame.MOUSEBUTTONDOWN):
            if(self.boxBody.collidepoint(event.pos)):  # 若按下鼠标且位置在文本框
                self.active = not self.active
            else:
                self.active = False
            self.color = self.color_active if(
                self.active) else self.color_inactive
        if(event.type == pygame.KEYDOWN):  # 键盘输入响应
            if(self.active):
                if(event.key == pygame.K_RETURN):
                    print(self.text)
                    # self.text=''
                elif(event.key == pygame.K_BACKSPACE):
                    self.text = self.text[:-1]
                else:
                    self.text += event.unicode

    def draw(self, screen: pygame.surface.Surface):
        txtSurface = self.font.render(
            self.text, True, self.color)  # 文字转换为图片
        width = max(120, txtSurface.get_width()+10)  # 当文字过长时，延长文本框
        self.boxBody.w = width
        screen.blit(txtSurface, (self.boxBody.x+5, self.boxBody.y+5))
        pygame.draw.rect(screen, self.color, self.boxBody, 2)
        
def starting_screen():
    screen.blit(bg, (0,0))

    game_title = font.render('Starting Screen', True, WHITE)
    end_title = font.render('Complete! Start in 5 s', True, WHITE)

    screen.blit(game_title, (display_width//2 - game_title.get_width()//2, 150))

    play_button = Button('Play', WHITE, None, 350, centered_x=True)
    exit_button = Button('Exit', WHITE, None, 400, centered_x=True)
    
    play_button.display()
    exit_button.display()
    
    pygame.display.set_caption('水排序')
    clock = pygame.time.Clock()
    pygame.display.update()
    
    running=True

    while running:
        
        clock.tick(60)

        if play_button.check_click(pygame.mouse.get_pos()):
            play_button = Button('Play', RED, None, 350, centered_x=True)
        else:
            play_button = Button('Play', WHITE, None, 350, centered_x=True)

        if exit_button.check_click(pygame.mouse.get_pos()):
            exit_button = Button('Exit', RED, None, 400, centered_x=True)
        else:
            exit_button = Button('Exit', WHITE, None, 400, centered_x=True)

        play_button.display()
        exit_button.display()
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                raise SystemExit
        if pygame.mouse.get_pressed()[0]:
            if play_button.check_click(pygame.mouse.get_pos()):
                running=False

            if exit_button.check_click(pygame.mouse.get_pos()):
                pygame.quit()
                sys.exit()
                raise SystemExit

    screen.blit(bg, (0,0))
    game_title = font.render('Starting Screen', True, WHITE)
    screen.blit(game_title, (display_width//2 - game_title.get_width()//2, 150))
    inputbox1 = InputBox(pygame.Rect(150, 400, 50, 32))
    inputbox2 = InputBox(pygame.Rect(400, 400, 50, 32))
    inputbox4 = InputBox(pygame.Rect(650, 400, 50, 32))
    inputbox5 = InputBox(pygame.Rect(900, 400, 50, 32))
    font1 = pygame.font.Font(font_addr, 18)
    ques1 = font1.render('nonempty num', True, WHITE)
    ques2 = font1.render('empty num', True, WHITE)
    ques4 = font1.render('color num', True, WHITE)
    ques5 = font1.render('difficulty', True, WHITE)
    screen.blit(ques1, (145, 360))
    screen.blit(ques2, (410, 360))
    screen.blit(ques4, (665, 360))
    screen.blit(ques5, (920, 360))
    gen_button = Button('generate', WHITE, None, 500, centered_x=True)
    gen_button.display()
    pygame.display.update()
    
    running=True
    while running:
        clock.tick(60)
        if gen_button.check_click(pygame.mouse.get_pos()):
            gen_button = Button('generate', RED, None, 500, centered_x=True)
        else:
            gen_button = Button('generate', WHITE, None, 500, centered_x=True)
        gen_button.display()
        pygame.display.update()
    
         
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                raise SystemExit
            inputbox1.dealEvent(event)  # 输入框处理事件   
            inputbox2.dealEvent(event)
            inputbox4.dealEvent(event)
            inputbox5.dealEvent(event)
            
        inputbox1.draw(screen)  # 输入框显示
        inputbox2.draw(screen) 
        inputbox4.draw(screen) 
        inputbox5.draw(screen) 
        pygame.display.update()
        
        if pygame.mouse.get_pressed()[0]:
            if gen_button.check_click(pygame.mouse.get_pos()):  
                nonempty_num=int(inputbox1.text)
                empty_num=int(inputbox2.text)
                colornum=int(inputbox4.text)
                difficulty=int(inputbox5.text)
                running=False
                
    screen.blit(bg, (0,0))            
    pygame.display.update()
    colordict={0:(18, 53, 85),1:(209,73, 78),2:(230,155,3),3:(219,208,167),4:(151,173,172),5:(255,150,128),6:(0,34,40)}
    env=environment(nonempty_num,empty_num,1,difficulty,colornum)
    env.initialize()
    
    #画图显示tube
    
    tubenum=nonempty_num+empty_num
    tubewidth=80
    interspace=30
    tubeheight=3*tubewidth
    left=display_width//2-(tubewidth*tubenum+interspace*(tubenum-1))/2
    bottom=400
    
    for tube in env.tubes:
        order=tube.order
        Vols=tube.Vols 
        print(order._data,Vols._data)
        pygame.draw.rect(screen,(0,0,0),(left,bottom-tubeheight,tubewidth,tubeheight),width=1)
        for i,j in zip(order._data,Vols._data):  
            color=colordict[i]
            top=bottom-tubeheight/env.Max_Vol*j
            Rect=(left,top,tubewidth,round(tubeheight/env.Max_Vol*j))
            pygame.draw.rect(screen,color,Rect,width=0)
            bottom=top
        left=left+interspace+tubewidth
        bottom=400
    pygame.display.update()
    
    running=True
    begin=False
    while running:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                raise SystemExit
            #设置每个tube的按钮
            if pygame.mouse.get_pressed()[0]:
                left=display_width//2-(tubewidth*tubenum+interspace*(tubenum-1))/2
                i=0
                check=True
                while check:
                    rect=pygame.Rect(left,bottom-tubeheight,tubewidth,tubeheight)
                    if rect.collidepoint(pygame.mouse.get_pos()):
                        if begin:
                            if first_tube==env.tubes[i]:
                                print('cannot chose',i)
                            else:
                                first_tube.pour_into(env.tubes[i])
                                begin=not begin
                                print('pour into',i)
                                #更新试管
                                screen.blit(bg, (0,0))            
                                left=display_width//2-(tubewidth*tubenum+interspace*(tubenum-1))/2
                                bottom=400
                                for tube in env.tubes:
                                    order=tube.order
                                    Vols=tube.Vols 
                                    print(order._data,Vols._data)
                                    pygame.draw.rect(screen,(0,0,0),(left,bottom-tubeheight,tubewidth,tubeheight),width=1)
                                    for i,j in zip(order._data,Vols._data):  
                                        color=colordict[i]
                                        top=bottom-tubeheight/env.Max_Vol*j
                                        Rect=(left,top,tubewidth,round(tubeheight/env.Max_Vol*j))
                                        pygame.draw.rect(screen,color,Rect,width=0)
                                        bottom=top
                                    left=left+interspace+tubewidth
                                    bottom=400
                                pygame.display.update()  

                        else :
                            begin=not begin
                            print('chose tube',i)
                            first_tube=env.tubes[i]
                        check=False
                    else:
                        left=left+interspace+tubewidth
                    i+=1
        sortednum=0
        for tube in env.tubes:
            if tube.issorted():
                sortednum+=1
        if sortednum==env.nonempty_num:
            screen.blit(end_title, (display_width//2 - end_title.get_width()//2, 500))
            pygame.display.update() 
            t1 =pygame.time.wait(5000)
            starting_screen()  
                
            

# screen = pygame.display.set_mode((display_width, display_height))
# bg = pygame.image.load(bg_location)
# font_addr = pygame.font.get_default_font()
# font = pygame.font.Font(font_addr, 48)

# #starting_screen()            

In [4]:
env=environment(4,2,1,3,4)

In [5]:
env.initialize()

pieces [0.020244722476618793, 0.9797552775233812] 0
pieces [1] 1
pieces [1] 2
pieces [0.7463515941500468, 0.2536484058499532] 3
adjoint: [0, 3, 1, 0, 2, 3] [0.9797552775233812, 0.2536484058499532, 1, 0.020244722476618793, 1, 0.7463515941500468]


In [6]:
env.game_cheat()

KeyboardInterrupt: 

In [7]:
env.anwser

AttributeError: 'environment' object has no attribute 'anwser'

In [11]:
    env=environment(nonempty_num,empty_num,1,difficulty,colornum)
    env.initialize()

pieces [0.7431961184473755, 0.22237942234630836, 0.034424459206316094] 0
pieces [0.5159134547810693, 0.4840865452189307] 1
pieces [0.021681319154886802, 0.9783186808451132] 2
pieces [0.5359553638422953, 0.10597003515777592, 0.3580746009999288] 3
adjoint: [1, 0, 3, 2, 1, 0, 3, 0, 2] [0.5159134547810693, 0.22237942234630836, 0.46404463615770475, 0.021681319154886802, 0.4840865452189307, 0.7431961184473755, 0.5359553638422953, 0.034424459206316094, 0.9783186808451132]


In [11]:
def search_for_answer(self):
    if self.status==0 or self.status==2:  ##第一次遇见这种环境
        current_solutions=self.solve_problem()   ## 求解目前所有可行解
        if current_solutions==[]:   ##找不到可行解
            sortednum=0
            for tube in self.tubes:
                if tube.issorted():
                    sortednum+=1
            if sortednum==self.nonempty_num:   ##判断胜利与否
                return(self.rec)    ##输出last step集合，即完全解法
            else:
                self.status=1  ##转换为状态1
                self.go_back() ##回到上一试管状态
                self.search_for_answer() ##继续下一步
        else:                 ##目前有解
            self.status=0   ##转换为状态0
            self.history.push(self.tubes) ### 记录此刻的状态,以供回溯
            self.solutions.push(current_solutions) #####记录该层全部可行解
            current_operation=current_solutions[0] #####执行最贪心的策略
            self.rec.push(current_operation)  ###记录决策
            for i in current_operation:
                self.pour(i)       ####执行最优解
            self.search_for_answer() ##继续下一步
    else:          ##通过回溯来到这个状态
        if self.rec.top()==self.solutions.top()[-1]: ######过去已经走过了该层所有决策
            self.status=3  ##转换为状态3
            self.history.remove_last_record() ###此状态以后不会再回来了
            self.solutions.remove_last_record()  ###此层可行解已用尽
            self.rec.remove_last_record()     ###此条路径已经完全否定
            self.go_back() ##回到上一试管状态
            self.search_for_answer() ##继续下一步
        else:   ####这层的可行解还没用完
            self.status=2 ##转换为状态2
            current_operation=self.solutions.top()[self.solutions.top().index(self.rec.top())+1]  ####索引移到次一位贪心，准备执行
            self.rec.push(current_operation) ###记录决策
            for i in current_operation:
                self.pour(i)       ####执行最优解
            self.search_for_answer() ##继续下一步

In [10]:
def go_back(self):
    if self.history.isEmpty():
        print('empty tube number:',self.empty_num,'is not enough')
        self.empty_num+=1
        self.added_empty_tubes+=1
        self.stats=0
        self.search_for_answer() 
    self.tubes=self.history.top() ####  试管重置到上一状态
    

In [9]:
def pour(from_to):
    self.tubes[from_to[0]].pour_into(self.tubes[from_to[1]])

In [7]:
a=(1,2)

In [8]:
a[0]

1

In [2]:
solutions=[[(1,2),(3,2)],[(3,4)],[(1,3)],[(2,3)],[(3,3)]]

In [3]:
solutions.index([(1,2),(3,2)])

0

#### 上一步输出的决策为1倒2，决策元组为(1,2)

In [27]:
decision=(1,2)

In [28]:
rec=ArrayStack([])

In [29]:
rec.push([decision])

In [30]:
rec.record_new_branch((2,3))

[[(1, 2), (2, 3)]]

In [32]:
rec.push([(1,2)])

In [33]:
rec._data

[[(1, 2), (2, 3)], [(1, 2)]]

In [34]:
rec.remove_last_record()

In [36]:
rec._data[-1]

[(1, 2), (2, 3)]

给定一个处境，有四种情况：

status=0 第一次到这种状况，需要求解当前可行解，存入下一层可行解集，并且第一个可行解，记录这次的last step

status=1 第一次到这种状况，但是无可行解，需要回溯，这层的可行解集无需存入，laststep无需更新

status=2 回溯上来，无需求解，根据目前的可行解集，和已经做过的last step，往下试一个，记录last step

status=3 回溯上来，无需求解，根据目前可行解集，last step是该层最后的元素，需要回溯，last step删除一个